## 1. POS Tagging:-

In [54]:
import nltk, re, pandas as pd
from nltk.corpus import stopwords
sent1 = 'The university was founded in 1885 by Leland and Jane Stanford in memory of their only child, Leland Stanford Jr., who had died of typhoid fever at age 15 the previous year. Stanford was a former Governor of California and U.S. Senator; he made his fortune as a railroad tycoon. The school admitted its first students on October 1, 1891,[2][3] as a coeducational and non-denominational institution.'
sent1

'The university was founded in 1885 by Leland and Jane Stanford in memory of their only child, Leland Stanford Jr., who had died of typhoid fever at age 15 the previous year. Stanford was a former Governor of California and U.S. Senator; he made his fortune as a railroad tycoon. The school admitted its first students on October 1, 1891,[2][3] as a coeducational and non-denominational institution.'

In [32]:
## Preprocessing:-
# sent = ' '.join([word for word in nltk.word_tokenize(re.sub(r'[.,;]', '', sent).replace('[2][3]','')) if word not in stopwords.words('english')])
# sent

In [60]:
sent2 = 'It was founded in 2002 by Elon Musk with the goal of reducing space transportation costs to enable the colonization of Mars'
pos_tag = nltk.pos_tag(nltk.word_tokenize(sent2))
print(pos_tag)

[('It', 'PRP'), ('was', 'VBD'), ('founded', 'VBN'), ('in', 'IN'), ('2002', 'CD'), ('by', 'IN'), ('Elon', 'NNP'), ('Musk', 'NNP'), ('with', 'IN'), ('the', 'DT'), ('goal', 'NN'), ('of', 'IN'), ('reducing', 'VBG'), ('space', 'NN'), ('transportation', 'NN'), ('costs', 'NNS'), ('to', 'TO'), ('enable', 'VB'), ('the', 'DT'), ('colonization', 'NN'), ('of', 'IN'), ('Mars', 'NNS')]


### 1.1 Find  tag pattern from pos:-
1. https://stackoverflow.com/questions/32399299/how-do-i-extract-patterns-from-lists-of-pos-tagged-words-nltk
2. https://stackoverflow.com/questions/34672986/detecting-pos-tag-pattern-along-with-specified-words

#### 1.1.1 By particuler tag pattern 
for it we have to know about the tage pattern for our document and information

In [42]:
# For three words:-
def match(l,p1,p2,p3):
    for sub in l:
        # avoid index error and catch last three elements
        end = len(sub) - 1
        for ind, (a, b) in enumerate(sub, 1):
            if ind == end:
                break
            if b == p1 and sub[ind][1] == p2 and sub[ind + 1][1] == p3:
                yield ("{} {} {}".format(a, sub[ind][0], sub[ind + 1][0]))

list_of_pos = [pos_tag]               
print(list(match(list_of_pos,"VBN","IN", "CD")))  ## For year(CD: cardinal digit) of(IN:preposition/subordinating conjunction) establishment(VBD: VERB PAST TENSE)

['founded in 2002']


In [50]:
def match(l,p1,p2,p3):
    for sub in l:
        # avoid index error and catch last three elements
        end = len(sub) - 1
        for ind, (a, b) in enumerate(sub, 1):
            if ind == end:
                break
            if b == p1 and sub[ind][1] == p2 and sub[ind + 1][1] == p3:
                yield ("{} {} {}".format(a, sub[ind][0], sub[ind + 1][0]))

list_of_pos = [pos_tag]               
print(list(match(list_of_pos,"NNP","CC","NNP")))

[]


In [48]:
# For two words:-
def match(l,p1,p2):
    for sub in l:
        # avoid index error and catch last three elements
        end = len(sub) - 1
        for ind, (a, b) in enumerate(sub, 1):
            if ind == end:
                break
            if b == p1 and sub[ind][1] == p2:
                yield ("{} {} {}".format(a, sub[ind][0], sub[ind + 1][0]))

list_of_pos = [pos_tag]               
print(list(match(list_of_pos,"DT","NN"))) 

['the goal of', 'the colonization of']


#### 1.1.2 Triple Method
a triple is a set of three entities that codifies a statement about semantic data in the form of subject–predicate[Modifiers]–object
expressions

In [44]:
## Who founded the university::-- Who wants a noun in reply and founded is a past verb so VBN
##                                reply may be:- founded by ....
temp = 0
for var in pos_tag: ## It gives the chunk of VBN to NNP
    if var[1] == 'VBN' or temp == 1:
        print(var[0], end = ' ')
        temp = 1
        if var[1] == 'NNP':
            print()
            temp = 0

founded in 2002 by Elon 


#### 1.2 Text information extraction and text analysis:-
1. https://www.kdnuggets.com/2018/08/named-entity-recognition-practitioners-guide-nlp-4.html

In [22]:
df = pd.DataFrame(pos_tag, columns = ['word', 'POS'])
df.head(3)

,word,POS
0,The,DT
1,university,NN
2,founded,VBD


In [23]:
### Let’s now find out the most frequent named entities in our news corpus
df = pd.DataFrame(df.groupby(['word', 'POS']).size()).reset_index().rename(columns = {0:'freq'})
df.sort_values(by = 'freq', ascending = False)[:10]

,word,POS,freq
11,Stanford,NNP,3
12,The,DT,2
8,Leland,NNP,2
0,1,CD,1
27,non-denominational,JJ,1
22,fortune,JJ,1
23,founded,VBD,1
24,institution,NN,1
25,made,VBD,1
26,memory,NN,1


In [24]:
# Most Frequent NNP = singular proper noun 
#               NNPS = plural proper noun 
df[(df['POS'] == 'NNP') | (df['POS'] == 'NNPS')]  

,word,POS,freq
4,California,NNP,1
5,Governor,NNP,1
6,Jane,NNP,1
7,Jr,NNP,1
8,Leland,NNP,2
9,October,NNP,1
10,Senator,NNP,1
11,Stanford,NNP,3
13,US,NNP,1


## 2.NER
https://medium.com/greyatom/learning-pos-tagging-chunking-in-nlp-85f7f811a8cb
#### 2.1 Noun pharse chunking  https://www.nltk.org/book/ch07.html

In [56]:
# NER:- 

grammar = ('''NP: {<DT>?<JJ>*<NN>} # NP''')
chunk_parser = nltk.RegexpParser(grammar)
result = chunk_parser.parse(nltk.pos_tag(nltk.word_tokenize(sent1)))

# print(result) # For print like mention in doc as tree
for i in result: # For print in a systemtical manner
    print(i)

(NP The/DT university/NN)
('was', 'VBD')
('founded', 'VBN')
('in', 'IN')
('1885', 'CD')
('by', 'IN')
('Leland', 'NNP')
('and', 'CC')
('Jane', 'NNP')
('Stanford', 'NNP')
('in', 'IN')
(NP memory/NN)
('of', 'IN')
('their', 'PRP$')
(NP only/JJ child/NN)
(',', ',')
('Leland', 'NNP')
('Stanford', 'NNP')
('Jr.', 'NNP')
(',', ',')
('who', 'WP')
('had', 'VBD')
('died', 'VBN')
('of', 'IN')
(NP typhoid/NN)
(NP fever/NN)
('at', 'IN')
(NP age/NN)
('15', 'CD')
(NP the/DT previous/JJ year/NN)
('.', '.')
('Stanford', 'NNP')
('was', 'VBD')
('a', 'DT')
('former', 'JJ')
('Governor', 'NNP')
('of', 'IN')
('California', 'NNP')
('and', 'CC')
('U.S', 'NNP')
('.', '.')
('Senator', 'NNP')
(';', ':')
('he', 'PRP')
('made', 'VBD')
('his', 'PRP$')
(NP fortune/NN)
('as', 'IN')
(NP a/DT railroad/NN)
(NP tycoon/NN)
('.', '.')
(NP The/DT school/NN)
('admitted', 'VBD')
('its', 'PRP$')
('first', 'JJ')
('students', 'NNS')
('on', 'IN')
('October', 'NNP')
('1', 'CD')
(',', ',')
('1891', 'CD')
(',', ',')
('[', 'VBD')
('2', 

#### 2.2 Relation extractor:-

In [10]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

def get_continuous_chunks(text):
        chunked = ne_chunk(pos_tag(word_tokenize(text)))
        continuous_chunk = []
        current_chunk = []
        for i in chunked:
            if type(i) == Tree:
                current_chunk.append(" ".join([token for token, pos in i.leaves()]))
            elif current_chunk:
                named_entity = " ".join(current_chunk)
                if named_entity not in continuous_chunk:
                    continuous_chunk.append(named_entity)
                    current_chunk = []
            else:
                 continue
        return continuous_chunk

In [12]:
get_continuous_chunks(sentence)

['Leland', 'Jane Stanford', 'Leland Stanford', 'California', 'U.S']

In [17]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

def get_continuous_chunks(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []

    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    if continuous_chunk:
        named_entity = " ".join(current_chunk)
        if named_entity not in continuous_chunk:
            continuous_chunk.append(named_entity)

    return continuous_chunk
get_continuous_chunks(sent)

['European', 'Google', '']

In [16]:
sent = 'European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices'
sent

'European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices'